<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Inventory Management
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Inventory management focuses on optimizing stock levels to meet customer demand while minimizing holding costs and reducing the risk of stockouts and overstock situations. Effective inventory management enables companies to maintain the right balance between supply and demand, ensuring products are available when needed without tying up capital in excess inventory.
<p style = 'font-size:16px;font-family:Arial;color:#00233C'></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The main objectives of Inventory Management are:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Demand Fulfillment:</b> Ensuring sufficient stock levels are maintained to meet customer demand across different products and locations, minimizing lost sales due to stockouts.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Cost Control:</b> Reducing costs associated with overstocking, such as storage and obsolescence, while optimizing replenishment and ordering processes to minimize stock holding costs.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Supply Chain Efficiency:</b> Streamlining processes between suppliers, warehouses, and retail locations to maintain a smooth supply chain flow and reduce lead times.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Inventory Accuracy:</b> Reducing discrepancies between recorded and actual stock levels, which can lead to phantom inventory and stockouts.</li><li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Data-Driven Insights:</b> Utilizing historical sales data, seasonal trends, and other demand indicators to forecast and manage inventory levels effectively, supporting long-term planning and profitability.</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> For effective decision-making, business analysts must analyze data to identify patterns, assess inventory health, and determine when corrective actions are needed in stores or warehouses. By providing a streamlined approach to managing data analysis and making insights readily accessible, we enable more proactive and informed inventory management, support sales growth, reduce waste, and improve overall operational efficiency.<br>
Teradata Vantage has a proven track record of performing complex calculations on vast datasets through its advanced parallel processing and scalability. Additionally, ClearScape Analytics offers powerful and flexible tools for attribution analysis, text processing, and statistical techniques that handle millions or even billions of data points. These analytics can be seamlessly combined with other analyses to create AI and ML pipelines directly within the database, eliminating the need to transfer large volumes of data. Thus, leveraging Teradata Vantage significantly enhances decision-making and optimizes strategic outcomes.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. </p>

In [ ]:
%connect local

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We will setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
Set query_band='DEMO=Inventory_Management_SQL.ipynb;' update for session;

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
call get_data('DEMO_Inventory_cloud'); ---takes about 10 seconds
---call get_data('DEMO_Inventory_local'); ---takes about 40 seconds

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
call space_report();  -- optional, takes about 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Data Exploration</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s begin by examining our data. We have two datasets: the inventory data, which contains transactional information about inventory, and the vendor data, which includes details about the vendors.</p>

In [ ]:
select top 5* from DEMO_Inventory.Inventory_Data;

In [ ]:
select top 5* from DEMO_Inventory.Vendor_Data;

In [ ]:
select min(Sale_Date) as start_date, max(Sale_Date) as end_date from DEMO_Inventory.Inventory_Data;

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 Missing Data</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As we have seen above that the minimum date for our inventory data is 2019-01-01 but not all the combinations of Store and Sku have data for each day. This is due to fact that inventory systems mostly log only the changes happening in the inventory. If there is no change there will be no record in the inventory_data table. Thus as the first step in our data preprocessing we'll make sure that our dataset has all the records.<br>For this we will use ClearScape's TD_Resample function which transforms an irregular time series to a regular time series by resampling the data at regular intervals. The function has various interpolation methods to fill in missing data points in the resampled time series.</p>

In [ ]:
select Store_Id,Sku,Product_Category,min(Sale_Date) as Start_Date, max(Sale_Date) as End_Date 
from DEMO_Inventory.Inventory_Data group by Store_Id,Sku,Product_Category;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As we can see from the above output not all the combinations of store, item and product category have the same starting date. For creating the regular series for each store, item and product category we will as a first step insert a record with 0 values so that we can subsequently generate a consistent, continuous series that includes all relevant records.</p>

In [ ]:
CREATE VOLATILE TABLE all_inv AS
    (
    SELECT *
        FROM DEMO_Inventory.Inventory_Data)WITH DATA
    ON COMMIT PRESERVE ROWS;

In [ ]:
--Insert record for start date when not present
INSERT INTO all_inv
    SELECT DATE '2019-01-01' AS Sale_Date,
        x.Store_Id,
        x.Sku,
        x.Product_Category,
        0 AS Total_Sales_Units,
        0 AS On_Hand_Inventory_Units,
        0 AS Replenishment_Units,
        0 AS Inventory_Pipeline,
        0 AS Units_In_Transit,
        0 AS Units_In_Dc,
        0 AS Units_On_Order,
        0 AS Units_Under_Promotion,
        0 AS Shelf_Capacity
        FROM (
        SELECT DISTINCT Store_Id,
            Sku,
            Product_Category
            FROM DEMO_Inventory.Inventory_Data 
            QUALIFY MIN(Sale_Date) OVER (PARTITION BY store_id, sku, product_category ) > '2019-01-01' )x;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let us use the TD_Resample function to create a record for each date and make our series a regular series.</p>

In [ ]:
/*create resampled inventory data */
EXECUTE FUNCTION INTO ART(Inv_Resampled)
    TD_RESAMPLE(
    SERIES_SPEC(
    TABLE_NAME(all_inv),
    ROW_AXIS(TIMECODE(Sale_Date)),
    SERIES_ID(store_id,sku,product_category),
    PAYLOAD(FIELDS(total_sales_units,
    on_hand_inventory_units ,
    replenishment_units ,
    inventory_pipeline ,
    units_in_transit ,
    units_in_dc ,
    units_on_order ,
    units_under_promotion ,
    shelf_capacity),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS(
    TIMECODE(
    START_VALUE(DATE '2019-01-01'),
    DURATION(CAL_DAYS(1))
    ),
    INTERPOLATE(LAG)
    ),
    OUTPUT_FMT(INDEX_STYLE(FLOW_THROUGH))
    );

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For better understanding we will see the data of one store and item.</p>

In [ ]:
select * from Inv_Resampled where store_id = 171 and sku = 64 order by Row_I;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us plot on_hand_inventory_units</p>

In [ ]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(DEMO_User.Inv_Resampled),
        ROW_AXIS(TIMECODE("ROW_I")),
        SERIES_ID(store_id,sku,product_category),
        PAYLOAD (FIELDS("On_hand_inventory_units"),CONTENT(REAL))
    )where store_id = 171 and sku = 64 ,
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('line'),
            LEGEND('upper left'),
            TITLE('Inventory Units for Store_id 171 and SKU 64')
        )],
        IMAGE('png')
    )
);

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Issues of Phantom Inventory</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Phantom inventory refers to inventory that appears in inventory systems as available but is not actually present in the physical stock. This discrepancy creates an illusion of higher stock levels than what’s really available, leading to problems such as:<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li>Stockouts: The system believes products are in stock, so new orders aren’t placed in time, leading to stockouts.</li><li>
Lost Sales and Customer Dissatisfaction: Customers may place orders expecting products that aren’t physically available, resulting in delays and frustration.</li><li>
Inefficient Replenishment: False stock levels cause issues in planning, leading to unnecessary orders or missed replenishments.</li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Phantom inventory issues often arise due to system errors, poor tracking, inaccurate manual counts, damage, shrinkage (theft, loss), or unprocessed returns. To catch this in the system we can create a phantom_inventory_flag which is set whenever the stock difference is significantly high. For our example we have taken a five times the average daily sales for that product.<br>For this the we have calculated below values and set the flag accordingly.<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li>
    <b>Daily Sales Calculation:</b> It calculates the average daily sales for each product in each store to understand typical sales patterns over time.</li><li><b>Starting Stock for the Day:</b>
For each day we take a look at the previous day's ending stock to set a starting point for today. If there’s no record from the previous day, it estimates the starting stock by adding today’s sales to today’s replenishments.</li><li><b>Ending Stock for the Day:</b>
This is the actual count of items in stock at the end of each day. If there’s no data, it assumes zero.</li><li><b>Phantom Inventory Calculation:</b>
Phantom inventory is calculated as the difference between expected stock at the end of the day and the actual end-of-day stock. Expected stock is based on yesterday’s leftover items plus any new items that arrived today, minus today’s sales. If these numbers don’t match up, it suggests a "phantom" difference that could indicate missing or unaccounted items.</li><li><b>Flagging Large Discrepancies:</b>
A flag is set if the phantom inventory (difference) is significant, specifically if it’s more than five times the average daily sales. This helps catch larger, unusual discrepancies, prompting further investigation.
In summary, this code flags situations where the difference between expected and actual stock is unusually high, helping identify potential inventory issues.</li></ul>

In [ ]:
/*CREATE Phantom Inventory */
CREATE MULTISET VOLATILE TABLE Phantom_Inventory AS
    (
    SELECT       
        x.ROW_I AS sales_date,
        x.store_id AS store_id,
        x.sku AS sku,
        x.daily_sales_units AS daily_sales_units,
        x.on_hand_inventory_units AS on_hand_inventory_units,
        x.start_on_hand_units AS start_on_hand_units,
        x.replenishment_units AS replenishment_units,
        x.total_sales_units AS total_sales_units,
        x.end_on_hand_units AS end_on_hand_units,
        x.phantom_inventory AS phantom_inventory,
        x.phantom_inventory_ind AS phantom_inventory_ind
        FROM (
        SELECT a.*,
            AVG(total_sales_units)OVER(PARTITION BY sku, store_id
            ORDER BY row_i) AS daily_sales_units,
                COALESCE(LAG(on_hand_inventory_units, 1) OVER(PARTITION BY store_id, sku
            ORDER BY row_i), 
            on_hand_inventory_units + total_sales_units - replenishment_units
            ) AS start_on_hand_units,
                COALESCE(on_hand_inventory_units, 0) AS end_on_hand_units,
                start_on_hand_units + replenishment_units - total_sales_units - end_on_hand_units AS phantom_inventory,
                
            CASE
                WHEN phantom_inventory <> 0 AND ABS(phantom_inventory) > 5 * daily_sales_units THEN 1 
            ELSE 0
            END AS phantom_inventory_ind
            FROM demo_user.Inv_Resampled a
        ) x )
    WITH DATA ON COMMIT PRESERVE ROWS;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us check the data for the store and item we have picked earlier.</p>

In [ ]:
select * from Phantom_Inventory where store_id = 171 and sku = 64 order by sales_date;

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Issues of Identify Out of Stocks Items</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Out-of-stock situations i.e when the item is not available for purchase, happen when there isn't enough stock to meet customer demand. To avoid this, retailers set a safety stock level, which acts as a warning point. When stock falls below this level, it signals that it’s time to reorder more. The remaining stock should then be enough to cover customer demand until the new stock arrives.<br>There are different ways to figure out the right safety stock level for a product, which is the amount of extra stock to keep on hand to avoid running out. In our demo we will look at two methods that fit the specific situation of each store and product:<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li><b>
    Previous Stock Before Reorders:</b> This approach uses the amount of stock that was on hand before previous reorder times.</li><li><b>Average Daily Sales and Delivery Time:</b> This method calculates safety stock based on how much the product usually sells each day and how long it takes for new stock to arrive after ordering.</li></ul>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
For each store and item, we will choose the smaller of these two values as the safety stock level. For this  we first we will need an estimate of the inventory on hand, which we worked on in the earlier phantom inventory calculations.</p>

In [ ]:
/*Identify Out of Stocks*/
CREATE MULTISET VOLATILE TABLE inv_pi AS (
    SELECT x.store_id AS store_id,
        x.sku AS sku,
        x.sales_date AS sales_date,
        x.product_category AS product_category,
        x.total_sales_units AS total_sales_units,
        x.on_hand_inventory_units AS on_hand_inventory_units,
        x.replenishment_flag AS replenishment_flag,
        x.replenishment_units AS replenishment_units,
        x.units_on_order AS units_on_order,
        x.units_in_transit AS units_in_transit,
        x.units_in_dc AS units_in_dc,
        x.phantom_inventory AS phantom_inventory,
         -- Calculate the running total of estimated on-hand inventory    
        GREATEST(
        LEAST(
        SUM(COALESCE(x.replenishment_units, 0) - COALESCE(x.total_sales_units, 0) - COALESCE(x.phantom_inventory, 0))
        OVER (PARTITION BY x.store_id, x.sku
        ORDER BY x.sales_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),
        x.on_hand_inventory_units
        ),
        0
        ) AS estimated_on_hand_inventory
        FROM (
        SELECT       
            inv.store_id AS store_id,
            inv.sku AS sku,
            inv.row_i AS sales_date,
            inv.product_category AS product_category,
            
            CASE
                WHEN inv.on_hand_inventory_units < 0 THEN 0 
            ELSE inv.on_hand_inventory_units
            END AS on_hand_inventory_units,
                inv.total_sales_units AS total_sales_units,
                
            CASE
                WHEN inv.replenishment_units > 0 THEN 1
            ELSE 0
            END AS replenishment_flag,
                
            CASE
                WHEN replenishment_flag = 1 THEN inv.replenishment_units
            ELSE 0
            END AS replenishment_units,
                inv.units_on_order AS units_on_order,
                inv.units_in_transit AS units_in_transit,
                inv.units_in_dc AS units_in_dc,
                COALESCE(pi.phantom_inventory, 0) AS phantom_inventory --0 as estimated_on_hand_inventory
            
            FROM demo_user.Inv_Resampled inv
            JOIN Phantom_Inventory pi
            ON inv.store_id = pi.store_id
            AND inv.sku = pi.sku
            AND inv.row_i = pi.sales_date
        )x
    ) WITH DATA ON COMMIT PRESERVE ROWS;

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have incorporated the below logic in the sql above:<br><b>
    Adjusting Inventory Values:</b> It ensures the on-hand inventory doesn’t fall below zero. If the current stock is negative, it’s set to zero to prevent unrealistic numbers. For replenishments we are using a replenishment_flag to indicate whether new units were added, setting it to 1 if there was a restock and 0 if not. Only when this flag is 1 does the replenishment count contribute to the stock.<b>Calculating Estimated On-Hand Inventory:</b>The code calculates an estimated_on_hand_inventory, which is a running total of the on-hand stock based on previous replenishments, sales, and phantom inventory adjustments. For this we are taking 
<br>Adding/Subtracting Stock Changes: It calculates changes to stock by adding replenishments, subtracting sales, and adjusting for phantom inventory.<br>
Setting Upper and Lower Bounds: It ensures that this estimated inventory doesn’t exceed the actual recorded on-hand units or fall below zero.<br><b>
    Joining Inventory and Phantom Data</b>:Finally, the inventory data (inv) and phantom inventory data (pi) are combined based on store, product, and sale date. This allows the table to have both real-time stock information and any phantom discrepancies for analysis.<br>
In summary, the sql allows us to track inventory changes over time, while controlling for discrepancies and setting boundaries to avoid unrealistic inventory values.

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Calculate Average On-Hand and Average Daily Sales Metrics</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For calculating safety stock, we will first calculate some key sales metrics. We calculate the average amount on-hand prior to a replenishment event as well as the average number of units sold for each store-item. For both metrics, we'll limit the calculation to the 90 days prior to the current inventory record.</p>

In [ ]:
/*inventory metrics */
CREATE MULTISET VOLATILE TABLE inv_metrics AS (
    SELECT y.store_id AS store_id,
        y.sku AS sku,
        y.sales_date AS sales_date,
        y.product_category AS product_category,
        y.on_hand_inventory_units AS on_hand_inventory_units,
        y.total_sales_units AS total_sales_units,
        y.replenishment_flag AS replenishment_flag,
        y.replenishment_units AS replenishment_units,
        y.units_on_order AS units_on_order,
        y.units_in_transit AS units_in_transit,
        y.units_in_dc AS units_in_dc,
        y.phantom_inventory AS phantom_inventory,
        y.estimated_on_hand_inventory AS estimated_on_hand_inventory,
        y.prior_inventory AS prior_inventory,
        y.rolling_stock_onhand AS rolling_stock_onhand,
        y.rolling_min_expected_stock AS rolling_min_expected_stock,
        COALESCE(LAST_VALUE(y.rolling_min_expected_stock) OVER(PARTITION BY y.store_id, y.sku
        ORDER BY y.sales_date),0)
        AS min_expected_stock,
            COALESCE(LAST_VALUE(y.avg_sales_units) OVER(PARTITION BY y.store_id, y.sku
        ORDER BY y.sales_date),0)
        AS daily_sales_units
        FROM (
        SELECT x.*,
            SUM(x.prior_inventory) OVER(PARTITION BY x.store_id, x.sku
            ORDER BY x.sales_date ROWS BETWEEN 90 PRECEDING AND CURRENT ROW) /
            (SUM(x.replenishment_flag) OVER(PARTITION BY x.store_id, x.sku
            ORDER BY x.sales_date ROWS BETWEEN 90 PRECEDING AND CURRENT ROW) + 1) 
            AS rolling_stock_onhand,
                
            CASE
                WHEN x.replenishment_flag <> 1 
            THEN 0
            ELSE COALESCE(rolling_stock_onhand,0)
            END AS rolling_min_expected_stock,
                AVG(x.total_sales_units) OVER(PARTITION BY x.store_id, x.sku
            ORDER BY x.sales_date ROWS BETWEEN 90 PRECEDING AND CURRENT ROW) AS avg_sales_units
            FROM (
            SELECT a.*,
                
                CASE
                    WHEN a.replenishment_flag = 1 
                THEN COALESCE(LAG(a.estimated_on_hand_inventory, 1) OVER(PARTITION BY a.store_id, a.sku
                ORDER BY a.sales_date),0)
                ELSE 0
                END AS prior_inventory
                FROM inv_pi a
            )x
        )y
    ) WITH DATA ON COMMIT PRESERVE ROWS;

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Calculate Safety Stock</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> Safety stock is a buffer of extra inventory kept on hand to help avoid running out of inventory due to fluctuations in demand or delays in restocking. It ensures that there is enough product available to meet unexpected demand or delays in replenishment, providing a safety net to maintain consistent availability.<br>Safety stock calculations consider factors like:<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li>Average demand</li>
    <li>Lead time for restocking</li><li>
    Variability in demand and lead time</li></ul><p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    In the below sql we get the shortest lead time for each Store-SKU based on the lead times for the replenishment of particular SKUs within a given store location. This will let us know the safety stock value for each Store-SKU. With this information calculated we can determine when the<br> On-hand inventory is less than the safety stock level -on_hand_less_than_safety_stock<br>The requested replinishment units are not sufficient to meet safety stock requirements -insufficient_inventory_pipeline_units<br>The inventory pipeline is one day away from not being able to fulfill stocking requirements -insufficient_lead_time    

In [ ]:
/* inventory safety stock */
CREATE MULTISET VOLATILE TABLE inv_safety_stock AS (
    SELECT
        inv.sales_date,
        inv.store_id,
        inv.sku,
        inv.product_category,
        inv.total_sales_units,
        inv.on_hand_inventory_units,
        inv.replenishment_units,
        inv.replenishment_flag,
        inv.phantom_inventory,
        inv.estimated_on_hand_inventory,
        inv.prior_inventory,
        inv.rolling_min_expected_stock,
        inv.min_expected_stock,
        inv.daily_sales_units,
        inv.daily_sales_units * ven.min_lead_time AS ss_sales_velocity,
        
        CASE
            WHEN inv.replenishment_flag <> 1 
        THEN 0 
        ELSE
        CASE
            WHEN inv.min_expected_stock < ss_sales_velocity 
        THEN min_expected_stock 
        ELSE ss_sales_velocity
        END
        END AS safety_stock,
            inv.units_on_order,
            inv.units_in_transit,
            inv.units_in_dc,
            COALESCE(ven.lead_time_in_transit,0) AS lead_time_in_transit,
            COALESCE(ven.lead_time_in_dc,0) AS lead_time_in_dc,
            COALESCE(ven.lead_time_on_order,0) AS lead_time_on_order,
            
        CASE
            WHEN estimated_on_hand_inventory <= safety_stock THEN 1 ELSE 0
        END AS on_hand_less_than_safety_stock,
            
        CASE
            WHEN  (on_hand_less_than_safety_stock = 1) AND 
        (inv.units_on_order + inv.units_in_transit + inv.units_in_dc <> 0) AND
        ((inv.units_on_order + inv.units_in_transit + inv.units_in_dc) < (safety_stock - estimated_on_hand_inventory))
        THEN 1
        ELSE 0
        END AS insufficient_inventory_pipeline_units,
            
        CASE
            WHEN on_hand_less_than_safety_stock = 1 AND
        (inv.units_on_order + inv.units_in_transit + inv.units_in_dc <> 0) AND
        ((inv.units_on_order + inv.units_in_transit + inv.units_in_dc) >= (safety_stock - estimated_on_hand_inventory)) 
        THEN GREATEST(
        COALESCE(lead_time_in_transit,0),
        COALESCE(lead_time_on_order,0),
        COALESCE(lead_time_in_dc,0)
        )+1
        ELSE NULL
        END  AS lead_time
        FROM inv_metrics inv
        LEFT JOIN (
        SELECT a.*,
            LEAST(lead_time_in_dc, lead_time_in_transit, lead_time_on_order) AS min_lead_time
            FROM demo_inventory.vendor_data a)ven
        ON inv.store_id = ven.store_id
        AND inv.sku = ven.item_id
    )WITH DATA ON COMMIT PRESERVE ROWS;

In [ ]:
CREATE MULTISET VOLATILE TABLE inv_ss_with_lead_times AS (
    SELECT store_id,
        sku,
        sales_date,
        lead_time,
        lead_date,
        prev_lead_time,
        day_diff,
        
        CASE
            WHEN (lead_time IS NULL AND (prev_lead_time - day_diff) <= 0) THEN 1
        ELSE 0
        END 
        AS insufficient_lead_time
        FROM(
        SELECT 
            a.store_id,
            a.sku,
            a.sales_date,
            MAX(a.lead_time) AS lead_time,
            MAX(b.sales_date) AS lead_date,
            MAX(b.lead_time) AS prev_lead_time,
            CAST(MIN((a.sales_date - b.sales_date)DAY(4)) AS BIGINT) AS day_diff
            FROM inv_safety_stock a
            LEFT JOIN inv_safety_stock b
            ON a.store_id=b.store_id 
            AND a.sku=b.sku 
            AND a.sales_date > b.sales_date
            WHERE b.lead_time IS NOT NULL
            GROUP BY 1,
                2,
                3 )x
    )WITH DATA ON COMMIT PRESERVE ROWS; 

In [ ]:
CREATE MULTISET VOLATILE TABLE lead_time_alerts AS (
    SELECT 
        inv_ss.sales_date,
        inv_ss.store_id,
        inv_ss.sku,
        inv_ss.product_category,
        inv_ss.total_sales_units,
        inv_ss.on_hand_inventory_units,
        inv_ss.replenishment_units,
        inv_ss.replenishment_flag,
        inv_ss.phantom_inventory,
        inv_ss.estimated_on_hand_inventory,
        inv_ss.prior_inventory,
        inv_ss.rolling_min_expected_stock,
        inv_ss.min_expected_stock,
        inv_ss.daily_sales_units,
        inv_ss.ss_sales_velocity,
        inv_ss.safety_stock,
        inv_ss.units_on_order,
        inv_ss.units_in_transit,
        inv_ss.units_in_dc,
        inv_ss.lead_time_in_transit,
        inv_ss.lead_time_in_dc,
        inv_ss.lead_time_on_order,
        inv_ss.on_hand_less_than_safety_stock,
        inv_ss.insufficient_inventory_pipeline_units,
        inv_ss.lead_time,
        inv_lead.insufficient_lead_time
        FROM inv_safety_stock inv_ss
        JOIN inv_ss_with_lead_times inv_lead
        ON inv_ss.store_id = inv_lead.store_id
        AND inv_ss.sku = inv_lead.sku
        AND inv_ss.sales_date = inv_lead.sales_date
    )WITH DATA ON COMMIT PRESERVE ROWS; 

In [ ]:
CREATE MULTISET VOLATILE TABLE out_of_stock_alerts AS (
    SELECT sales_date,
        store_id,
        sku,
        product_category,
        total_sales_units,
        daily_sales_units,
        
        CASE
            WHEN (on_hand_less_than_safety_stock = 1 AND insufficient_inventory_pipeline_units = 1) THEN 1
            WHEN (on_hand_less_than_safety_stock = 1 AND insufficient_inventory_pipeline_units <> 1 AND insufficient_lead_time = 1 )
        THEN 1
        ELSE 0
        END 
        AS oos_alert_indicator
        FROM lead_time_alerts
    )WITH DATA ON COMMIT PRESERVE ROWS;

In [ ]:
select * from out_of_stock_alerts where store_id = 171 and sku = 64 order by sales_date;

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Issues of Identify Zero Sales</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Not all products generate sales daily at each store location. However, when a product remains unsold for an extended period, it may indicate a potential issue with its inventory status, warranting verification.The challenge lies in defining what constitutes an "extended period" since sales patterns differ widely across products. For example:<br><b>High-demand products</b>: For items that consistently sell hundreds of units each day, a sudden zero-sales day could signal an inventory issue.<br><b>
    Low-demand products:</b> Conversely, for products that typically sell only a few units per month, multiple days or even weeks without sales may not be cause for concern.<br>
To identify potential inventory issues effectively, we assess the frequency of zero-sales days for each store-SKU combination relative to the total days the product has been available for sale. This approach allows us to estimate the likelihood of a zero-sales day for each product, facilitating a data-driven perspective on which products may require inventory review. 

In [ ]:
/* # calculate ratio of total days of zero sales and total days on shelf across observed period */
CREATE MULTISET VOLATILE TABLE zero_sales_totals AS (
    SELECT 
        store_id,
        sku,
        SUM(
        CASE
            WHEN total_sales_units = 0 THEN 1 ELSE 0
        END) AS total_zero_sales_days,
            COUNT(*) AS total_days,
            CAST(total_zero_sales_days AS DECIMAL(22,8)) / total_days   AS zero_sales_day_probability
        FROM demo_user.Inv_Resampled
        GROUP BY store_id,
            sku
    ) WITH DATA ON COMMIT PRESERVE ROWS;   

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.1 Calculate Consecutive Zero Sales Days</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We calculate the consecutive days a product has experienced zero-units sold</p>

In [ ]:
CREATE MULTISET VOLATILE TABLE zero_sales_days  AS (
    SELECT
        sales_date,
        store_id,
        sku,
        total_sales_units,
        zero_sales_flag_rank,
        sales_change_flag_inv,
        
        CASE
            WHEN total_sales_units <> 0 THEN 0 
        ELSE SUM(sales_change_flag_inv) OVER(PARTITION BY store_id, sku, zero_sales_flag_rank
        ORDER BY sales_date)
        END
        AS total_days_wo_sales
        FROM (
        SELECT 
            sales_date,
            store_id,
            sku,
            total_sales_units,
            SUM(sales_change_flag) OVER(PARTITION BY store_id, sku
            ORDER BY sales_date ) AS zero_sales_flag_rank,
                sales_change_flag_inv
            FROM (
            SELECT
                row_i AS sales_date,
                store_id,
                sku,
                total_sales_units,
                
                CASE
                    WHEN total_sales_units=0 AND LAG(total_sales_units,1) OVER(PARTITION BY store_id, sku
                ORDER BY row_i) <> 0 THEN 1 
                ELSE 0
                END AS sales_change_flag,
                    
                CASE
                    WHEN total_sales_units = 0 THEN 1 ELSE 0
                END  AS sales_change_flag_inv
                FROM demo_user.Inv_Resampled
            ) x)y)
    WITH DATA ON COMMIT PRESERVE ROWS;   

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.2 Calculate Cumulative Probability of Zero Sales Event</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we know the probability that a product will not sell on any given day in a specific store, we can estimate the likelihood that it will experience multiple consecutive days without sales. This calculation involves a simple probability function that evaluates the chance of back-to-back zero-sales days.<br>
When this calculated probability reaches or exceeds a certain threshold—in this case, 5%, which indicates a low tolerance for risk—we might consider this a signal to check the product’s inventory. The threshold serves as a trigger point, helping to ensure that inventory checks are focused on situations with a significant likelihood of being problematic.</p>

In [ ]:
CREATE MULTISET VOLATILE TABLE zero_sales_inventory   AS (
    SELECT 
        a.sales_date,
        a.store_id,
        a.sku,
        a.total_sales_units,
        a.zero_sales_flag_rank,
        a.sales_change_flag_inv,
        a.total_days_wo_sales,
        power(CAST(zero_sales_day_probability AS DECIMAL(22,8)), total_days_wo_sales) AS zero_sales_probability,
        
        CASE
            WHEN zero_sales_probability < 0.05 THEN 1 ELSE 0
        END AS no_sales_flag
        FROM zero_sales_days a
        LEFT JOIN zero_sales_totals b
        ON a.store_id = b.store_id
        AND a.sku = b.sku
    )WITH DATA ON COMMIT PRESERVE ROWS;    

In [ ]:
select * from zero_sales_inventory where store_id = 171 and sku = 64 order by sales_date;

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
In the above demo we have seen how Vantage can solve the complex business problems having multiple calculations and logic involving huge datasets.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time. This section drops all the tables created during the demonstration.</p>

In [ ]:
Drop Table Inv_Resampled;

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_Inventory');          -- Takes 5 seconds

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Inventory Data:</b>

- `Sale_Date ` : Date of the sale
- `Store_Id ` : Unique Id of store
- `Sku ` : Item
- `Product_Category ` : Category in which item belongs
- `Total_Sales_Units ` : Total item units sold
- `On_Hand_Inventory_Units ` : Number of item units on hand in store
- `Replenishment_Units ` : Number of item units for replenishment
- `Inventory_Pipeline ` : Number of item units in pipeline
- `Units_In_Transit ` : Number of item units in transit
- `Units_In_Dc ` : Number of item units in distribution center
- `Units_On_Order ` : Number of item units ordered
- `Units_Under_Promotion ` : Number of item units in sales promotion
- `Shelf_Capacity` : Storage capacity of item units in the store

<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Vendor Data:</b>

- `Id ` : Unique Id
- `Vendor_Id ` : Unique Id of Vendor
- `Sub_Vendor_Id ` : Unique Id of Sub Vendor
- `Store_Id ` : Store id which vendor caters to
- `Item_Id ` : Id of Item, SKU
- `Lead_Time_In_DC ` : Lead time for item to reach Distribution Center
- `Lead_Time_In_Transit ` : Lead time for item in transit
- `Lead_Time_On_Order` : Lead time for item on order placed

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023, 2024. All Rights Reserved
        </div>
    </div>
</footer>